In [10]:
# Import necessary packages
import os
import rioxarray as rxr
# import earthpy as et
import pandas as pd 

# Reference: https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/fundamentals-raster-data/intro-to-the-geotiff-file-format/


In [4]:
datatif = rxr.open_rasterio("R-Factor-CONUS/R-Factor_CONUS.tif",
                                 masked=True)

In [5]:
datatif

<xarray.DataArray (band: 1, y: 3677, x: 5874)> Size: 86MB
[21598698 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 47kB -2.398e+06 -2.398e+06 ... 2.299e+06 2.3e+06
  * y            (y) float64 29kB 3.209e+06 3.208e+06 ... 2.69e+05 2.682e+05
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:       Area
    STATISTICS_MAXIMUM:  717.80529785156
    STATISTICS_MEAN:     134.98527006927
    STATISTICS_MINIMUM:  0
    STATISTICS_STDDEV:   143.78544244658
    scale_factor:        1.0
    add_offset:          0.0

In [8]:
datatif.rio.bounds()

(-2398837.623373987, 267782.5626054434, 2300362.376626013, 3209382.5626054434)

In [9]:
# View generate metadata associated with the raster file
print("The crs of your data is:", datatif.rio.crs)
print("The nodatavalue of your data is:", datatif.rio.nodata)
print("The shape of your data is:", datatif.shape)
print("The spatial resolution for your data is:", datatif.rio.resolution())
print("The metadata for your data is:", datatif.attrs)

The crs of your data is: EPSG:5070
The nodatavalue of your data is: nan
The shape of your data is: (1, 3677, 5874)
The spatial resolution for your data is: (800.0, -800.0)
The metadata for your data is: {'AREA_OR_POINT': 'Area', 'STATISTICS_MAXIMUM': 717.80529785156, 'STATISTICS_MEAN': 134.98527006927, 'STATISTICS_MINIMUM': 0, 'STATISTICS_STDDEV': 143.78544244658, 'scale_factor': 1.0, 'add_offset': 0.0}


In [11]:
df = pd.DataFrame(datatif)


ValueError: Must pass 2-d input. shape=(1, 3677, 5874)

In [14]:
datatif


<xarray.DataArray (band: 1, y: 3677, x: 5874)> Size: 86MB
array([[[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]]], dtype=float32)
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 47kB -2.398e+06 -2.398e+06 ... 2.299e+06 2.3e+06
  * y            (y) float64 29kB 3.209e+06 3.208e+06 ... 2.69e+05 2.682e+05
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:       Area
    STATISTICS_MAXIMUM:  717.80529785156
    STATISTICS_MEAN:     134.98527006927
    STATISTICS_MINIMUM:  0
    STATISTICS_STDDEV:   143.78544244658
    scale_factor:        1.0
    add_offset:          0.0